# Preparo l'ambiente di esecuzione

Questa fase deve essere eseguita solo una volta per assicurarsi di avere tutto il necessario per far eseguire i vari script.

## Codice

Viene scaricata solo la parte di codice necessaria a far funzionare il testing 
dei modelli di PyTorch dalla repo di GitHub

In [ ]:
!rm -rf *

# installa git-lfs per scaricare il progetto
!apt install git-lfs
!git lfs install

!git clone https://github.com/ncvescera/TirocinioAI.git

!mkdir dataset

!mv ./TirocinioAI/code/torch/* ./
!mv ./TirocinioAI/code/analyze.py ./
!mv ./TirocinioAI/code/dataset_adapter ./
!mv ./TirocinioAI/datasets/aline.zip ./dataset

!rm -rf TirocinioAI/
!rm *.ipynb

## Controllo installazione di PyTorch

Importando i pacchetti `torch` e `torchvision.models`, se non ho alcun errore, so che sono presenti e funzionanti



In [ ]:
import torch
import torchvision.models

## Dataset

Per scaricare il dataset da kaggle seguire questa guida per abilitare Colab al download dei file da kaggle: [Guida](https://www.kaggle.com/general/74235)

Una volta ottenuto il file `kaggle.json` eseguire i comandi seguenti per caricarlo su Colab e spostarlo nella giusta destinazione.

In [ ]:

# permette di scegliere il file da caricare
from google.colab import files
files.upload()

# sposta il file nella giusta cartella
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/

# scarica il dataset e lo estrae
!kaggle datasets download -d ifigotin/imagenetmini-1000
!mkdir dataset
!unzip imagenetmini-1000.zip -d dataset/
!rm imagenetmini-1000.zip

# prepara il dataset ALine
!unzip ./dataset/aline.zip -d ./dataset/
!rm -rf ./dataset/aline.zip

# avvia lo script per adattare il dataset
!sed -i 's/dataset_path =.*/dataset_path = "..\/dataset\/originali"/' ./dataset_adapter/adapter.py
%cd ./dataset_adapter/
!python adapter.py

# sposta il dataset nella giusta cartella
%cd ..
!mv ./dataset_adapter/ALine ./dataset/

# Esecuzione del Testing

Configuro lo script con la giusta posizione del dataset e avvio i vari test






## Configurazione path dataset

**IMPORTANTE**, prima di far partire lo script assicurati di aver modificato la variabile `dataset_dir` all'interno del file `main.py` con il giusto percorso della cartella contenente le immagini !!

Un possibile esempio:

```
...
dataset_dir = './dataset/imagenet-mini/val'
...
```

Questo comando dovrebbe farlo in automatico, ma per sicurezza controllare sempre !

In [29]:
# sostituisce solo la prima occorrenza
!sed -i '0,/dataset_dir =.*/s/dataset_dir =.*/dataset_dir = ".\/dataset\/imagenet-mini\/val"/' main.py
!sed -i '0,/aline =.*/s/aline =.*/aline = ".\/dataset\/ALine"/' main.py

## Testing ImageNetMini

### Immagini Originali

Lo script può funzionare anche in modalità `single-thread` tramite l'argomento `--nothreads`.
Se si vuole avviare lo scritp in modalità `single-thread` decommentare la prima riga e commentare la seconda.

Avviare il testing con il seguente comando: 

In [ ]:
# !python main.py --nothreads
!python main.py

!mkdir results_imagenet
!mv *.csv results_imagenet/

### Immagini GrayScale

Per avviare il testing del dataset applicando il filtro `GrayScale` passare allo script il seguente argomento: `-g`.

Avviare il testing con il filtro GrayScale con il seguente comando:

In [ ]:
# !python main.py -g --nothreads
!python main.py -g

!mkdir gresults_imagenet
!mv *.csv gresults_imagenet/

### Analisi dei Risultati

Con il seguente comando vengono analizzati tutti i file csv creati durante la fase di testing e vengono ritornate le statistiche deli vari modelli

Risultati del testing con **immagini non alterate**:

In [ ]:
!python analyze.py $(ls -d ./results_imagenet/*)

Risultati del testing con **immagini con flitro GrayScale**:

In [ ]:
!python analyze.py $(ls -d ./gresults_imagenet/*)

## Testing ALine

### Immagini Originali

Lo script può funzionare anche in modalità `single-thread` tramite l'argomento `--nothreads`.
Se si vuole avviare lo scritp in modalità `single-thread` decommentare la prima riga e commentare la seconda.

Avviare il testing con il seguente comando: 

In [ ]:
# !python main.py --nothreads --dataset ALine
!python main.py --dataset ALine

!mkdir results_aline
!mv *.csv results_aline/

### Immagini GrayScale

Per avviare il testing del dataset applicando il filtro `GrayScale` passare allo script il seguente argomento: `-g`.

Avviare il testing con il filtro GrayScale con il seguente comando:

In [ ]:
# !python main.py -g --nothreads --dataset ALine
!python main.py -g --dataset ALine

!mkdir gresults_aline
!mv *.csv gresults_aline/

### Analisi dei Risultati

Con il seguente comando vengono analizzati tutti i file csv creati durante la fase di testing e vengono ritornate le statistiche deli vari modelli

Risultati del testing con **immagini non alterate**:

In [ ]:
!python analyze.py $(ls -d ./results_aline/*)

Risultati del testing con **immagini con flitro GrayScale**:

In [ ]:
!python analyze.py $(ls -d ./gresults_aline/*)